<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_blackhole_info_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_blackhole_info.py

Full training pipeline for BlackHoleInfoAI:

1. Synthetic dataset of 6 black hole parameters → 3 information‐recovery metrics
2. Physics‐informed residuals enforcing toy Page curve and entropy bounds
3. MLP with LayerNorm & Dropout for robustness
4. MC‐Dropout inference for uncertainty quantification
5. Training loop with AdamW, ReduceLROnPlateau, early stopping
6. Visualizations: training curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Black Hole Information Dataset
# ------------------------------------------------------------------------------
class BlackHoleInfoDataset(Dataset):
    def __init__(self, n_samples=8000, seed=123):
        np.random.seed(seed)
        # Inputs: mass (M☉), temperature (K), spin (a*), charge (Q),
        # peak_freq (Hz), hawking_flux (J/s)
        M  = np.random.uniform(5.0, 50.0, (n_samples,1)).astype(np.float32)
        T  = np.random.uniform(1e5, 1e7, (n_samples,1)).astype(np.float32)
        a  = np.random.uniform(0.0, 0.99,(n_samples,1)).astype(np.float32)
        Q  = np.random.uniform(0.0, 0.1, (n_samples,1)).astype(np.float32)
        f  = np.random.uniform(1e15, 1e17,(n_samples,1)).astype(np.float32)
        φ  = np.random.uniform(1e20,1e22,(n_samples,1)).astype(np.float32)

        X_raw = np.hstack([M, T, a, Q, f, φ])

        # Toy targets:
        # retention_frac ≈ exp(-T/M)
        rf = np.exp(-T / (M * 1e6))
        # scrambling_time ≈ M * (1 + a^2)
        st = M * (1 + a**2)
        # page_time_frac ≈ 0.5 + Q
        pt = 0.5 + Q
        Y_raw = np.hstack([rf, st, pt]).astype(np.float32)
        Y_raw += 0.02 * np.random.randn(*Y_raw.shape).astype(np.float32)

        # Normalize
        self.X_mean, self.X_std = X_raw.mean(0), X_raw.std(0) + 1e-6
        self.Y_mean, self.Y_std = Y_raw.mean(0), Y_raw.std(0) + 1e-6

        self.X = (X_raw - self.X_mean) / self.X_std
        self.Y = (Y_raw - self.Y_mean) / self.Y_std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )

# ------------------------------------------------------------------------------
# 2. Model Definition
# ------------------------------------------------------------------------------
class BlackHoleInfoAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dims=(128, 128), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics‐Informed Residual
# ------------------------------------------------------------------------------
def physics_residual(pred, inp, stats):
    # Denormalize
    X_den = inp * stats['X_std'] + stats['X_mean']
    Y_den = pred * stats['Y_std'] + stats['Y_mean']
    rf_pred, st_pred, pt_pred = Y_den.t()
    M, T, a, Q, f, φ = X_den.t()

    # Toy Page‐curve constraint: retention_frac ≈ exp(-T/(M*1e6))
    rf_true = torch.exp(-T / (M * 1e6))
    res1 = nn.MSELoss()(rf_pred, rf_true)

    # Scrambling time: M*(1 + a^2)
    st_true = M * (1 + a**2)
    res2 = nn.MSELoss()(st_pred, st_true)

    # Page time fraction: 0.5 + Q
    pt_true = 0.5 + Q
    res3 = nn.MSELoss()(pt_pred, pt_true)

    return res1 + res2 + res3

def total_loss(pred, true, inp, stats, λ=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, inp, stats)
    return mse + λ * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC‐Dropout Inference
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(x))
    arr  = torch.stack(preds, dim=0)
    mean = arr.mean(0)
    std  = arr.std(0)
    return mean, std

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_ld, val_ld, stats, device,
          lr=1e-3, wd=1e-5, λ=1.0, max_epochs=200, patience=20):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=0.5, patience=5)
    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for epoch in range(1, max_epochs + 1):
        # Training
        model.train()
        train_loss = 0.0
        for xb, yb in train_ld:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, λ)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_ld.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_ld:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, λ)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_ld.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train: {train_loss:.4e} | Val: {val_loss:.4e}")

        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_blackhole_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    model.load_state_dict(torch.load("best_blackhole_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training Curve")
    plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.xlabel(f"True {name}")
    plt.ylabel(f"Pred {name}")
    plt.title(name)
    plt.show()

def plot_uncertainty_heatmap(model, stats_np, device):
    # Vary mass vs temperature, fix others
    m_vals = np.linspace(5.0, 50.0, 100)
    t_vals = np.linspace(1e5, 1e7, 100)
    M_grid, T_grid = np.meshgrid(m_vals, t_vals)
    grid = np.zeros((M_grid.size, 6), dtype=np.float32)
    grid[:,0], grid[:,1] = M_grid.ravel(), T_grid.ravel()
    for i in (2,3,4,5):
        grid[:,i] = stats_np['X_mean'][i]

    Xn = torch.from_numpy((grid - stats_np['X_mean']) / stats_np['X_std']) \
             .float().to(device)
    _, std = mc_dropout_predict(model, Xn, T=100)
    std_rf = std[:,0].cpu().numpy().reshape(M_grid.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(M_grid, T_grid, std_rf, cmap='viridis', shading='auto')
    plt.colorbar(label='Std(retention_frac)')
    plt.xlabel('Mass (M☉)')
    plt.ylabel('Temperature (K)')
    plt.title('Uncertainty Heatmap: Retention Fraction')
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset & stats
    ds = BlackHoleInfoDataset(8000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device),
        'X_std' : torch.tensor(ds.X_std,  device=device),
        'Y_mean': torch.tensor(ds.Y_mean, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  device=device),
    }
    stats_np = {
        'X_mean': ds.X_mean, 'X_std': ds.X_std,
        'Y_mean': ds.Y_mean, 'Y_std': ds.Y_std,
    }

    # Split & loaders
    n_val = int(0.2 * len(ds))
    tr, va = random_split(ds, [len(ds) - n_val, n_val])
    tr_ld = DataLoader(tr, batch_size=128, shuffle=True)
    va_ld = DataLoader(va, batch_size=256)

    # Model, train, visualize
    model   = BlackHoleInfoAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).float().to(device)
    with torch.no_grad():
        Yp = model(X_all).cpu().numpy()
    Yt = ds.Y
    for i, name in enumerate(['retention_frac', 'scrambling_time', 'page_time_frac']):
        plot_scatter(Yt[:,i], Yp[:,i], name)

    plot_uncertainty_heatmap(model, stats_np, device)